# Sudoku solver

### 1.Implementation

In [1]:
import numpy as np
##import pandas as pd

### Load or create a sudoku puzzle
Let's suppose we are given a table with size 9*9, 

In [2]:
# print the result as form of sudoko
def print_sudoku(board):
    """Prints the Sudoku board in a readable format."""
    for row in board:
        print(" ".join(str(num) if num is not None else "." for num in row))

In [3]:
# Example Sudoku board (easy difficulty)
sudoku_easy= [
    [5, 3, None, None, 7, None, None, None, None],
    [6, None, None, 1, 9, 5, None, None, None],
    [None, 9, 8, None, None, None, None, 6, None],
    [8, None, None, None, 6, None, None, None, 3],
    [4, None, None, 8, None, 3, None, None, 1],
    [7, None, None, None, 2, None, None, None, 6],
    [None, 6, None, None, None, None, 2, 8, None],
    [None, None, None, 4, 1, 9, None, None, 5],
    [None, None, None, None, 8, None, None, 7, 9]
]

print_sudoku(sudoku_easy)

5 3 . . 7 . . . .
6 . . 1 9 5 . . .
. 9 8 . . . . 6 .
8 . . . 6 . . . 3
4 . . 8 . 3 . . 1
7 . . . 2 . . . 6
. 6 . . . . 2 8 .
. . . 4 1 9 . . 5
. . . . 8 . . 7 9


In [4]:
# Example Sudoku board (medium difficulty)
sudoku_medium = [
    [None, 2, None, 6, None, 8, None, None, None],
    [5, 8, None, None, None, 9, 7, None, None],
    [None, None, None, None, 4, None, None, None, None],
    [3, 7, None, None, None, None, 5, None, None],
    [6, None, None, None, None, None, None, None, 4],
    [None, None, 8, None, None, None, None, 1, 3],
    [None, None, None, None, 2, None, None, None, None],
    [None, None, 9, 8, None, None, None, 3, 6],
    [None, None, None, 3, None, 6, None, 9, None],
]

print_sudoku(sudoku_medium)

. 2 . 6 . 8 . . .
5 8 . . . 9 7 . .
. . . . 4 . . . .
3 7 . . . . 5 . .
6 . . . . . . . 4
. . 8 . . . . 1 3
. . . . 2 . . . .
. . 9 8 . . . 3 6
. . . 3 . 6 . 9 .


In [5]:
def if_valid(table, row, col, num):
    # check if number if valid within the row and column
    for i in range(9):
      if table[row-1][i]==num or table[i][col-1]==num:
         return False
    # check if valid in the 3*3 square
    start_row, start_col = 3 * ((row-1) // 3), 3 * ((col-1) // 3)
    for i in range(3):
        for j in range(3):
            if table[start_row + i][start_col + j] == num:
                return False
    return True

In [7]:
# Find possible digits for each position
def get_candidates(table, row, col):
    # candidates denotes in position (row,col) the possible candidates that valids
    candidates=[]

    for num in range(1,10):
        if if_valid(table, row, col, num)==True:
            candidates.append(num)

    return candidates


In [9]:
# Loop of filling in the only candidate, and find out the row and col of which filled in
def fill_sole_candidate(table):
    changed=True

    while changed:
        changed=False
        for row in range(1,10):
            for col in range(1,10):
                if table[row-1][col-1]==None:
                    candidates=get_candidates(table,row,col)
                    # check if candidates only contain one digit
                    if len(candidates)==1:
                        table[row-1][col-1]=candidates[0]
                        changed=True
                    

    return table

#### If the puzzle is an easy one, then with the following methhod it can be solved in pretty short time(less than 0.1s!)

In [10]:
#table=sudoku_easy
table=sudoku_easy
fill_sole_candidate(table)
print_sudoku(table)

5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9


#### Although for harder sudoku, it's totally different!

In [ ]:
#table=sudoku_easy
table=sudoku_medium
fill_sole_candidate(table)
print_sudoku(table)

. 2 . 6 . 8 . . .
5 8 . . . 9 7 . .
. . . . 4 . . . .
3 7 . . . . 5 . .
6 . . . . . . . 4
. . 8 . . . . 1 3
. . . . 2 . . . .
. . 9 8 . . . 3 6
. . . 3 . 6 . 9 .


#### Since by simply applying the only candidate rules does not work, we need more advanced techniques to solve the harder sudoku. 

#### First consider the standard backtracking method: by given a not filled sudoku table

In [12]:
# Compute possible candidates for each empty cell and store them in a dictionary.
def get_all_candidates(table):
    candidates_dict = {}
    for row in range(1,10):
        for col in range(1,10):
            if table[row-1][col-1] is None:  # Only check empty positions
                candidates_dict[(row, col)] = get_candidates(table, row, col)
    return candidates_dict

In [13]:
# Find the empty cell with the least candidates
def find_best_empty_cell(candidates_dict):
    return min(candidates_dict, key=lambda pos: len(candidates_dict[pos]))

In [14]:
# Solve the sudoku puzzle using fill sole candidate and guess candidates
def solve_sudoku(table):
    table = fill_sole_candidate(table)  # Apply Constraint Propagation
    candidates_dict = get_all_candidates(table)  # Get updated candidates after pre-filling

    if not candidates_dict:
        return True  # Sudoku is already solved

    # Find the empty cell with the fewest candidates (MRV heuristic)
    row, col = find_best_empty_cell(candidates_dict)

    for num in sorted(candidates_dict[(row, col)]):  # Try numbers in ascending order
        table[row-1][col-1] = num  # Place the number
        if solve_sudoku(table):  # Recursive call
            return True
        table[row-1][col-1] = None  # Backtrack

    return False  # No solution found


In [17]:
print("Original Sudoku:")
print_sudoku(sudoku_medium)

if solve_sudoku(sudoku_medium):
    print("\nSolved Sudoku:")
    print_sudoku(sudoku_medium)
else:
    print("No solution exists.")

Original Sudoku:
. 2 . 6 . 8 . . .
5 8 . . . 9 7 . .
. . . . 4 . . . .
3 7 . . . . 5 . .
6 . . . . . . . 4
. . 8 . . . . 1 3
. . . . 2 . . . .
. . 9 8 . . . 3 6
. . . 3 . 6 . 9 .
No solution exists.


## 2. Using a ILP(Integer Linear Program) to solve sudoku puzzle

In [18]:
import pandas as pd
from solve_sudoku_ilp import solve_sudoku_ilp

In [19]:
# Solve Sudoku
solved_sudoku = solve_sudoku_ilp(sudoku_medium)

# Display the solved Sudoku board
if solved_sudoku:
    df_solved_sudoku = pd.DataFrame(solved_sudoku)
    print(df_solved_sudoku)
else:
    print("No solution found.")


No solution found.
